In [1]:
import os
directory = os.getcwd()
sample_folder = '\\res'
res_folder = '\\res\\trn'
os.chdir(directory+res_folder)

In [2]:
import numpy as np
import pandas as pd
test = os.listdir()
avl_labels_og = [] # avaliable labels in the 
for i in test:
    if '_temp_flow' in i:
        prefix = i[:-14]
    elif '_control_signal' in i:
        prefix = i[:-19]
    elif '_energy' in i:
        prefix = i[:-11]
    else:
        continue
    avl_labels_og.append(prefix)
print(len(avl_labels_og))        
avl_labels_og = np.array(avl_labels_og)
avl_labels_og = np.unique(avl_labels_og)
print(len(avl_labels_og))

3699
1233


In [3]:
import pandas as pd

# compare with current_list and flag the ones that have been completely missed
# Add these values to the repeat dataframe
# To check with indiecs of current list, labels have to be converted to numeric values, especially labels with _cp
if len(avl_labels_og)==0:
    starting_label=0
else:
    existing_res = pd.read_csv('..\\sim_results.csv')
    starting_label = existing_res.index[-1]+1
    
repeat = pd.DataFrame()
current = pd.read_csv('..\\current_list.csv', header=0, index_col=0)
#current = pd.read_csv('..\\redo.csv', header=0, index_col=0)
current_index = [str(i+starting_label) for i in current.index]

avl_labels = np.array([''.join(filter(str.isdigit, s)) for s in avl_labels_og])

missing = list(set(current_index)-set(avl_labels))
print(missing)

for i in missing:
    repeat = repeat.append(current.loc[int(i)-starting_label])
repeat.head()

corrected_index = [i+starting_label for i in repeat.index]
repeat.index = corrected_index

repeat

[]


""


In [34]:
# if redo.cs is read, the labels can be converted to string. But the postfix _cp has to 
# added to the ones with design_case cp
# then the missing values labels are removed from the redo labels list
read_file = '..\\redo.csv'
inp = pd.read_csv(read_file, header=0,index_col=0)

if 'redo' in read_file:
    redo_labels = []
    for i in inp.index:
        if 'cp' in inp['design_case'][i]:
            redo_labels.append(str(i)+'_cp')
        else:
            redo_labels.append(str(i))
        avl_labels_og = list(set(redo_labels)-set(missing))
len(avl_labels_og)

12

In [49]:
repeat.index.min()

1043

In [7]:
# check length of all files and flag the onces that are incomplete

read_file = 'list_of_inputs.csv'
#read_file = '..\\redo.csv'
inp = pd.read_csv(read_file, header=0,index_col=0)

for i in avl_labels_og:
    #print(i+'_control_signal.txt')
    df = pd.read_csv(i+'_control_signal.txt', delimiter=',',index_col=0) 
    df2 = pd.read_csv(i+'_temp_flow.txt', delimiter=',',index_col=0) 
    df3 = pd.read_csv(i+'_energy.txt', delimiter=',',index_col=0) 
    if (len(df) <87601 or len(df2) <87601 or len(df3) <87601) :
        print(i + ' len = '+str(len(df)))
        j = ''.join([char for char in i if char.isdigit()])
        repeat = repeat.append(inp.loc[int(j)])
repeat

1023 len = 77666


ParserError: Error tokenizing data. C error: Expected 57 fields in line 26581, saw 67


In [53]:
# create repeat dataframe from current_list
avl_labels = np.array([''.join(filter(str.isdigit, s)) for s in avl_labels_og])
labels_all = list(range(861,1232))
labels_all = [str(i) for i in labels_all]
labels = [i for i in labels_all if i in avl_labels]
og_labels = []
for alo in avl_labels_og:
     if any(alo.startswith(l) for l in labels):
        og_labels.append(alo)
        
read_file = '..\\current_list.csv'
repeat2 = pd.DataFrame()
inp = pd.read_csv(read_file, header=0,index_col=0)
for i in og_labels:
    #print(i+'_control_signal.txt')
    df = pd.read_csv(i+'_control_signal.txt', delimiter=',',index_col=0) 
    df2 = pd.read_csv(i+'_temp_flow.txt', delimiter=',',index_col=0) 
    df3 = pd.read_csv(i+'_energy.txt', delimiter=',',index_col=0) 
    if (len(df) <87601 or len(df2) <87601 or len(df3) <87601) :
        print(i + ' len = '+str(len(df)))
        j = ''.join([char for char in i if char.isdigit()])
        repeat2 = repeat2.append(inp.loc[int(j)-starting_label])

repeat2.index = [i-starting_label for i in repeat2.index]
repeat = pd.concat([repeat,repeat2])

1023 len = 77666
1108 len = 84943
1110 len = 76635
1111 len = 77575
1112 len = 29411
1116_cp len = 24499
1117_cp len = 18089


In [6]:
# create repeat dataframe from redo list
redo = pd.read_csv('..\\redo.csv',index_col=0)
redo_index = redo.index
labels = []
for i in redo.index:
    if redo['design_case'].loc[i]=='cp_PV':
        labels.append(str(i)+'_cp')
    else:
        labels.append(str(i))

repeat2 = pd.DataFrame()
for i in labels:
    df = pd.read_csv(i+'_control_signal.txt', delimiter=',',index_col=0) 
    df2 = pd.read_csv(i+'_temp_flow.txt', delimiter=',',index_col=0) 
    df3 = pd.read_csv(i+'_energy.txt', delimiter=',',index_col=0) 
    if (len(df) <87601 or len(df2) <87601 or len(df3) <87601) :
        print(i + ' len = '+str(len(df)))
        j = ''.join([char for char in i if char.isdigit()])
        repeat2 = repeat2.append(redo.loc[int(j)])

In [9]:
len(labels)

123

In [19]:
#trial of difference using sets

a = ['1','2','3','4','5_cp','6_cp']
a2 = np.array([''.join(filter(str.isdigit, s)) for s in a])
b = ['1','2','3','4','5','7']

list(set(b)-set(a2))

['7']

In [61]:
repeat.to_csv('..\\redo.csv',index_label='label')

In [26]:
# make labels in the repeat dataframe, to delete the input files from the directories
labels = []
for i in repeat.index:
    if repeat.loc[i]['design_case']== 'cp_PV':
        labels.append(str(i)+'_cp')
    else:
        labels.append(str(i))
labels

['482',
 '729',
 '733',
 '766',
 '791',
 '794',
 '795',
 '808',
 '815',
 '853',
 '854',
 '856']

In [29]:
# delete the input files that correspond with repeat labels
house_folder = '..\\..\\house_and_backup'
backup_folder = '..\\..\\house_and_backup\\backup'

hfiles = os.listdir(house_folder)
bfiles = os.listdir(backup_folder)

# Iterate through the files and delete files with numbers from the list
for file in hfiles:
    for number in labels:
        if number in file:
            file_path = os.path.join(house_folder, file)
            os.remove(file_path)

for file in bfiles:
    for number in labels:
        if number in file:
            file_path = os.path.join(backup_folder, file)
            os.remove(file_path)

In [55]:
# check length of file from missed sims list and flag the onces that are incomplete
os.chdir(directory+res_folder)
import pandas as pd
repeat = pd.DataFrame()
inp = pd.read_csv('..\\redo.csv', header=0, index_col=0)
for i in inp.index:
    df = pd.read_csv(str(i)+'_control_signal.txt', delimiter=',',index_col=0) 
    if len(df) <87601 :
        print(str(i) + ' len = '+str(len(df)))
        #j = ''.join([char for char in i if char.isdigit()])
        repeat = repeat.append(inp.loc[int(i)])
repeat

""


In [55]:
all_files = os.listdir('..')
sample_files = []
for file in all_files:
    if 'sample' in file:
        sample_files.append(file)
sample_files

['lhs_sample_1.csv',
 'morris_sample_batt1.csv',
 'morris_sample_cp.csv',
 'morris_sample_cp2.csv',
 'morris_sample_cp3.csv',
 'morris_sample_pvt.csv',
 'morris_sample_pvt2.csv',
 'morris_sample_pvt3.csv',
 'morris_sample_pvt4.csv',
 'morris_sample_st.csv',
 'morris_sample_st2.csv',
 'morris_sample_st3.csv',
 'morris_sample_st4.csv']

In [42]:
slice_list = [356]
df2 = dfnew.iloc[dfnew.index.isin(slice_list)]
df2